# Извлечение признаков из текста

Этот блокнот разделён на две части:
* Сначала мы выясним, что нам понадобится для построения инструментария NLP (Natural Language Processing), который превратит набор текста в числовой массив из *признаков*. Для этого мы вручную вычислим, как часто встречаются те или иные слова, и построим метрику TF-IDF.
* Далее мы выполним эти шаги с помощью scikit-learn.

# Часть 1: основные принципы извлечения признаков из текста


С помощью базовых операций Python построим очень простую систему NLP. Возьмём *набор документов* (*corpus of documents*) - это будет два текстовых файла. Далее создадим *словарь* (*vocabulary*) из всех слов обоих документов. И затем посмотрим на технику *Мешок слов* (*Bag of Words*) для извлечения признаков из каждого документа.<br>
В этом файле приведены иллюстрации основных принципов!

## Начнаю с документов:
Для простоты изложения в  текстовых файлах One.txt и Two.txt не будет каких-либо знаков пунктуации. Откроем эти файлы и прочитаем данные. 

In [4]:
with open('One.txt') as mytext:
    print(mytext.read())

This is a story about dogs
our canine pets
Dogs are furry animals



In [5]:
with open('Two.txt') as mytext:
    print(mytext.read())

This story is about surfing
Catching waves is fun
Surfing is a popular water sport



### Читаю весь текст как строку string

In [6]:
with open('One.txt') as mytext:
    entire_text = mytext.read()

In [7]:
entire_text

'This is a story about dogs\nour canine pets\nDogs are furry animals\n'

In [8]:
print(entire_text)

This is a story about dogs
our canine pets
Dogs are furry animals



### Читаю каждую строку файла отдельно и помещаем в список

In [9]:
with open('One.txt') as mytext:
    lines = mytext.readlines()

In [10]:
lines

['This is a story about dogs\n',
 'our canine pets\n',
 'Dogs are furry animals\n']

### Читаю отдельные слова

In [11]:
with open('One.txt') as f:
    words = f.read().lower().split()

In [12]:
words

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

## Создаю словарь vocabulary (Мешок слов - "Bag of Words")

Для этого беру все слова из обоих документов, нахожу уникальные слова, и нумерую их.

In [13]:
with open('One.txt') as f:
    words_one = f.read().lower().split()

In [14]:
words_one

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

In [15]:
len(words_one)

13

In [16]:
uni_words_one = set(words)

In [17]:
uni_words_one

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'dogs',
 'furry',
 'is',
 'our',
 'pets',
 'story',
 'this'}

**То же самое для файла Two.txt**

In [18]:
with open('Two.txt') as f:
    words_two = f.read().lower().split()
    uni_words_two = set(words_two)

In [19]:
uni_words_two

{'a',
 'about',
 'catching',
 'fun',
 'is',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

**Получаем все уникальные слова из всех документов**

In [20]:
all_uni_words = set()
all_uni_words.update(uni_words_one)
all_uni_words.update(uni_words_two)

In [21]:
all_uni_words

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'catching',
 'dogs',
 'fun',
 'furry',
 'is',
 'our',
 'pets',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

In [22]:
full_vocab = dict()
i = 0

for word in all_uni_words:
    full_vocab[word] = i
    i = i+1

In [23]:
# Эти слова могут быть НЕ в алфавитном порядке! 
# Цикл for вовсе НЕ обязан проходит по множеству set() в алфавитном порядке!
full_vocab

{'this': 0,
 'furry': 1,
 'water': 2,
 'dogs': 3,
 'waves': 4,
 'surfing': 5,
 'popular': 6,
 'are': 7,
 'animals': 8,
 'a': 9,
 'about': 10,
 'canine': 11,
 'is': 12,
 'sport': 13,
 'story': 14,
 'fun': 15,
 'our': 16,
 'catching': 17,
 'pets': 18}

## Мешок слов -  как часто встречаются отдельные слова

После того, как был создан словарь из слов,  выполняется *извлечение признаков* для каждого из двух исходных документов:

**Создаю пустые счётчики для каждого документа**

In [24]:
one_freq = [0]*len(full_vocab)
two_freq = [0]*len(full_vocab)
all_words = ['']*len(full_vocab)

In [25]:
one_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [26]:
two_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
all_words

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [28]:
for word in full_vocab:
    word_ind = full_vocab[word]
    all_words[word_ind] = word    

In [29]:
all_words

['this',
 'furry',
 'water',
 'dogs',
 'waves',
 'surfing',
 'popular',
 'are',
 'animals',
 'a',
 'about',
 'canine',
 'is',
 'sport',
 'story',
 'fun',
 'our',
 'catching',
 'pets']

**Добавляю счётчики слов для каждого документа:**

In [31]:

with open('One.txt') as f:
    one_text = f.read().lower().split()
    
for word in one_text:
    word_ind = full_vocab[word]
    one_freq[word_ind]+=1

In [32]:
one_freq

[2, 2, 0, 4, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2]

In [33]:
# То же самое для второго документа:
with open('Two.txt') as f:
    two_text = f.read().lower().split()
    
for word in two_text:
    word_ind = full_vocab[word]
    two_freq[word_ind]+=1

In [34]:
two_freq

[1, 0, 1, 0, 1, 2, 1, 0, 0, 1, 1, 0, 3, 1, 1, 1, 0, 1, 0]

In [36]:
import pandas as pd
pd.DataFrame(data=[one_freq,two_freq],columns=all_words)

,this,furry,water,dogs,waves,surfing,popular,are,animals,a,about,canine,is,sport,story,fun,our,catching,pets
0,2,2,0,4,0,0,0,2,2,2,2,2,2,0,2,0,2,0,2
1,1,0,1,0,1,2,1,0,0,1,1,0,3,1,1,1,0,1,0


Некоторые слова есть в обоих документах, а некоторые слова есть только в `One.txt`, и некоторые другие слова есть только в `Two.txt`. Если применять эту логику для десятков тысяч документов, то  словарь вполне сможет вырасти до десятков тысяч слов. При этом матрицы будут содержать очень много нулей - это будут **разреженные метрицы** (**sparse matrices**).


# Продолжение:

## Мешок слов и Tf-idf
В приведённом выше примере, каждый вектор можно рассматривать как мешок слов (*bag of words*). Сами по себе эти вектора не очень полезны, но мы также можем добавить к ним частоту слов (*term frequencies - TF*) - как часто то или иное слово встречается в документе. Простой способ сделать это - это посчитать, сколько раз встречается слово в документе, и разделить на общее количество слов в документе. Тогда можно сравнивать между большими и маленькими документами, сколько раз то или иное слово встречается в документе.

Однако, если какое-то слово встречается во многих документах, то такое слово не будет являться хорошим признаком для отделения документов друг от друга. Чтобы работать с такими словами, можно добавить метрику *inverse document frequency (IDF)*, которая вычисляется как общее количество документов, разделить на количество документов, в которых содержится рассматриваемое нами слово (слово, для которого вычисляется IDF). В практических задачах это значение приводится к логарифмической шкале.

Соединяя метрики TF (Term Frequency) и IDF (Inverse Document Frequency), мы получаем метрику 

## Стоп-слова и морфология слов
Некоторые слова встречаются слишком часто, например слова "the" и "and" в английском языке. Эти слова можно просто исключить из рассмотрения. 

Кроме этого, одно и то же слово может встречаться в единственном и множественном числе, а также в различных падежах. Нам бы хотелось записать это слово один раз, а не записывать различные его вариации (например, записать только `cat` для обоих слов `cat` и `cats`). Это позволит нам уменьшить размер словаря и увеличить скорость работы модели.

С другой стороны, различные падежи слов - особенно в русском языке - несут дополнительную смысловую информацию. Эту информацию по-хорошему можно сохранять, то есть ссылаться на базовое слово в словаре, плюс сохранять дополнительные атрибуты этого слова в конкретном месте текста. В нашем блокноте мы ограничимся лишь базовым подсчётом частоты слов - как в отдельном документе, так и в целом наборе документов.

# Часть 2: извлечение признаков и текста в Scikit-Learn

Применяю sklearn для решения этих задач!

# Варианты извлечения признаков в Scikit-Learn

In [37]:
text = ['This is a line',
           "This is another line",
       "Completely different line"]

## CountVectorizer

In [38]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer

In [39]:
cv = CountVectorizer()

In [40]:
cv.fit_transform(text)

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [41]:
sparse_mat = cv.fit_transform(text)

In [42]:
sparse_mat.todense()

matrix([[0, 0, 0, 1, 1, 1],
        [1, 0, 0, 1, 1, 1],
        [0, 1, 1, 0, 1, 0]], dtype=int64)

In [43]:
cv.vocabulary_

{'this': 5, 'is': 3, 'line': 4, 'another': 0, 'completely': 1, 'different': 2}

In [44]:
cv = CountVectorizer(stop_words='english')

In [45]:
cv.fit_transform(text).todense()

matrix([[0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]], dtype=int64)

In [46]:
cv.vocabulary_

{'line': 2, 'completely': 0, 'different': 1}

## TfidfTransformer

TfidfVectorizer применяется для текстовых документов, а TfidfTransformer применяется к матрице со счётчиками, которую возвращает CountVectorizer

In [47]:
tfidf_transformer = TfidfTransformer()

In [48]:
cv = CountVectorizer()

In [49]:
counts = cv.fit_transform(text)

In [50]:
counts

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [51]:
tfidf = tfidf_transformer.fit_transform(counts)

In [52]:
tfidf.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

In [53]:
from sklearn.pipeline import Pipeline

In [54]:
pipe = Pipeline([('cv',CountVectorizer()),('tfidf',TfidfTransformer())])

In [55]:
results = pipe.fit_transform(text)

In [56]:
results

<3x6 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [57]:
results.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

## TfIdfVectorizer

Выполняет оба действия единым шагом!

In [58]:
tfidf = TfidfVectorizer()

In [59]:
new = tfidf.fit_transform(text)

In [60]:
new.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])